# **On Dispatching Design Patters**

If you have experience with Object Oriented Programming languages such as Python, you might
find strange the way packages are implemented in Julia. For example, Julia does not have classes!
Hence, you might wonder how you should design your code, since you probably won't be creating
classes with methods, and so on.

The multiple-dispatch capability of Julia makes design patterns in Julia quite different. First of all,
in Julia, instead, of classes we only have structs.
The struct can be thought of as composite "types", on which
functions can be written in order to transform such struct. Besides structs, we have abstract types,
which provide a hierarchy to the many types in Julia, and help to properly dispatch function calls.

Hence, a struct is not like a class where one defines methods,
but it's more like a "concrete" type to which functions are applied to.

## The Basics on Structs and Dispatch

Again, while in OOP a class will generate objects that have "methods" inside of it.
In Julia, there are no methods, it's all functions.
These functions are separated from the structs, and
the way we tie them together is via multiple dispatch.

An example might make things clearer. At first, let's create a struct to store 2D coordinate points.


In [1]:
using Pkg
Pkg.activate(".")

struct Point2D
    x::Real
    y::Real
end

p = Point2D(1, 1);

  Activating project at `~/MEGA/EMAP/Julia_Tutorials/Dispatch-Design`


The structs in Julia are by default immutable. This allows for more efficiency. Yet, if one needs,
the struct can be made mutable, by simply writing `mutable struct`.
Note also that whenever we define a struct, it already comes with a constructor function (e.g. `Point2D()`).
But this can be modified, as we are going to show shortly.

We can access the properties of a struct with the `p.x` notation, or using `getproperty(p, :x)`.
If you wish to know what are the properties of an object, just do `propertynames(p)`.

Next, let's expand on our example. Suppose we are going to work with geometric shapes.
Each struct will be an specific shape, such as a square, a rectangle, a triangle, and so on.
The functions will take shapes and perform some action.

Let's define the abstract type `Shape` which will be the "supertype"
of each geometric shape. This is useful when we have functions that work for every geometric shape,
independent of which shape we have. For example, we can define a function called
`totalarea` which is defined as the sum of individual ares for many shape and we can leave the definition of the `area`
for the geometric shapes themselves.

In [2]:
abstract type Shape end

Next, let's create some instances of shapes. 

In [3]:
mutable struct Square <: Shape
    center::Point2D
    length::Real
    Square(center, length) = length ≥ 0 ? new(center, length) : error("length should be greater or equal than 0.")
end

mutable struct Rectangle <: Shape
    center::Point2D
    height::Real
    width::Real
    Rectangle(center, height, width) = height ≥ 0 && width ≥ 0 ? new(center, height, width) : error("height and width should be greater or equal than 0.")
end

mutable struct Circle <: Shape
    center::Point2D
    radius::Real
    Circle(center, radius) = radius ≥ 0 ? new(center, radius) : error("radius should be greater or equal than 0.")
end

⬛ = Square(p, 10);
⚫ = Circle(p, 10);

The `new()` is a special function that allows one to construct the object from within the struct. We need to do so 
because the struct is not yet defined, hence the need for the `new`.

Once we defined our abstract type and our concrete types (the structs), we can dive in the dispatching. As we can see,
the shapes themselves do not have methods. What we do instead is that we define the function that acts on each
object based on it's type, hence "multiple dispatch".

In [4]:
height(s::Rectangle) = s.height
width(s::Rectangle)  = s.width

height(s::Square) = s.length
width(s::Square)  = s.length

area(s::Shape)    = width(s) * height(s)

area(s::Circle) = π * s.radius^2

totalarea(shapes::Shape...) = sum([area(s) for s in shapes])

@show totalarea(⚫,⬛);

totalarea(⚫, ⬛) = 414.1592653589793


Note the interesting thing that is happening in the code above. We've defined an `area` function that works on every shape. Yet,
a circle does not have a width and a height, so the function would not work for it. We then defined another `area` function
that dispatches on circles. Since `Circle` is a subtype of `Shape`, Julia's multiple dispatch gives precedence to the lowest type in the
hierarchy when calling a function. Thus, the code above works flawlessly.

This type of behavior in Julia enables us to come up with different code designs.

## Parametric Types

Let's create structs that collect the different shapes.

In [5]:
struct ShapeCollection{T <: Shape}
    list::Array{T}
end

⬛ = Square(Point2D(10,10), 10);
□  = Square(Point2D(0,0), 1);

⚫ = Circle(Point2D(0,0), 10)
◍ = Circle(Point2D(0,0), 4)

squares = ShapeCollection([⬛, □])
circles = ShapeCollection([⚫, ◍])
mix     = ShapeCollection([⬛, ⚫])

@show squares.list;
@show circles.list;
@show typeof(squares);
@show typeof(circles);
@show typeof(mix);
println("---------------------------------------")

squares.list = Square[Square(Point2D(10, 10), 10), Square(Point2D(0, 0), 1)]
circles.list = Circle[Circle(Point2D(0, 0), 10), Circle(Point2D(0, 0), 4)]
typeof(squares) = ShapeCollection{Square}
typeof(circles) = ShapeCollection{Circle}
typeof(mix) = ShapeCollection{Shape}
---------------------------------------


Note that when we created our struct, we added this `{T <: Shape}`.
This is specifying that a `ShapeCollection` will be a parametric type `T` where `T` is a subtype of `Shape`. In our example above,
our `squares` is of type `ShapeCollection{Squres}`, while the collection containing a mix of shapes, we get the type `ShaperCollection{Shape}`.

If we try to define a `ShapeCollection` with either a mix of shapes, or with things that are not shapes,
then we'll get an error.

In [6]:
nonshape = ShapeCollection([1,2])

LoadError: MethodError: no method matching ShapeCollection(::Vector{Int64})
[0mClosest candidates are:
[0m  ShapeCollection([91m::Array{T}[39m) where T<:Shape at In[5]:2

## Using Default Parameters and Units

In many situations, we might want our structs to have default values that could be
altered when creating a new instance. The `Parameters.jl` package makes this very easy to do.


In [7]:
using Parameters
@with_kw struct Board
    pencil::String = "black"
    background::String = "white"
    boardtype::String = "whiteboard"
    size::Tuple{Real,Real} = (1,1)
end

@show board = Board()
@show chalkboard = Board(pencil="white", background="green", boardtype="chalkboard");
println("----------------------------------------------------------------------------")

board = Board() = Board
  pencil: String "black"
  background: String "white"
  boardtype: String "whiteboard"
  size: Tuple{Int64, Int64}

chalkboard = Board(pencil = "white", background = "green", boardtype = "chalkboard") = Board
  pencil: String "white"
  background: String "green"
  boardtype: String "chalkboard"
  size: Tuple{Int64, Int64}

----------------------------------------------------------------------------


The struct above is great, but it could be improved. Note for example that it's possible in this struct to
declare a board with negative size, which is nonsense. Also, the size has no units, which makes it meaningless.
We could add another field to state the units, but there is a much more elegant way using the amazing
`Unitful.jl` package.

The `Parameters.jl` allow us to easily check whether the `size` is greater or equal than zero, without
needing to create a new constructor as we did in the previous examples.

In [8]:
using Unitful

@with_kw struct NewBoard
    pencil::String = "black"
    background::String = "white"
    boardtype::String = "whiteboard"
    size::Tuple{Unitful.Length,Unitful.Length} = (100u"cm",100u"cm"); @assert size[1] ≥ 0u"cm" && size[2] ≥ 0u"cm"
end

@show newboard = NewBoard();
println("----------------------------------------------------------------------------")

newboard = NewBoard() = NewBoard
  pencil: String "black"
  background: String "white"
  boardtype: String "whiteboard"
  size: Tuple{Quantity{Int64, 𝐋, Unitful.FreeUnits{(cm,), 𝐋, nothing}}, Quantity{Int64, 𝐋, Unitful.FreeUnits{(cm,), 𝐋, nothing}}}

----------------------------------------------------------------------------


This is nice, but `Unitful.jl` enables us to control this in a much more secure manner.
Suppose, for example, that we wish to send this specs to an american woodworker. Hence,
we need to convert the units for our table.

In [9]:
uconvert.(u"ft",newboard.size)

(1250//381 ft, 1250//381 ft)

Instead of creating an abstract type. We can also create types via unions, e.g. `Boards = Union{Board, NewBoard}`.

In [10]:
Boards = Union{Board, NewBoard}
function priceboard(b::T; comission=0.1, kwargs...) where T <: Boards
    return price(b::T; kwargs...)*(1 + comission)
end

function price(b::Board; units)
    units == "m" ? nothing : error("Size must be in m for pricing.")
    return b.boardtype == "whiteboard" ? reduce(*,b.size, init=10.5) : reduce(*,b.size, init=10.3)
end

function price(b::NewBoard)
    s = uconvert.(u"m",b.size)
    v = uconvert(Unitful.NoUnits,reduce(*,s)/1u"m^2")
    return b.boardtype == "whiteboard" ? 10.5v : 10.3v
end

priceboard(newboard) == priceboard(board, units="m"); # Returns true

In the example above, we created a function `priceboard` that works on both `Board` and `NewBoard` type. Note 
that we used here this `where T <: Boards`. This is similar to what we did for structs, but we use this `where` notation
when we wish to make functions parametric. Note that depending on the type of the board, a different `price` function
will be used.

### Fields Inheritance

Here is an example of a struct with fields that inherit from others.
The `color` and the `style`fields are working as "optional" fields, that may be used if the
user wants to set all the "subfields" at once.

In [86]:
@with_kw struct My_Struct
    color::Union{String,Nothing} = nothing
    color1::String = color === nothing ? "white" : color
    color2::String = color === nothing ? "black" : color
    
    style::Union{String,Nothing} = nothing
    style1::String = style === nothing ? "blue"  : style
    style2::String = style === nothing ? "white" : style
    
    width::Real  = 10; @assert width > 0
    height::Real = 10; @assert height > 0
    
    area = width * height
end

@show My_Struct(color1="blue")
@show My_Struct(color="yellow", style="none", width = 1);

My_Struct(color1 = "blue") = My_Struct
  color: Nothing nothing
  color1: String "blue"
  color2: String "black"
  style: Nothing nothing
  style1: String "blue"
  style2: String "white"
  width: Int64 10
  height: Int64 10
  area: Int64 100

My_Struct(color = "yellow", style = "none", width = 1) = My_Struct
  color: String "yellow"
  color1: String "yellow"
  color2: String "yellow"
  style: String "none"
  style1: String "none"
  style2: String "none"
  width: Int64 1
  height: Int64 10
  area: Int64 10



This idea of creating "optional" field could also be done with multiple dispatch, but the solution **does not work**
if you create your struct using `Parameters.jl`.

In [93]:
struct Example
    color1::String
    color2::String
end

Example(;kwargs...) = Example(values(kwargs))
Example(nt::NamedTuple{(:color,)}) = Example(nt.color, nt.color)

@show Example(color="red")
@show Example("black", "white");

Example(color = "red") = Example("red", "red")
Example("black", "white") = Example("black", "white")


### Extending the Dispatch Functionality

The multiple dispatch functionality in Julia is not only beautiful, but also powerful. Yet, it only
works for variables of different types. It would be nice if we could use the dispatch in other situations, such
as depending on wheter a value is larger than 0, or if the user provided an specific keyword.
Of course, we can always write many "if" and "else" statements to deal with it.
But the dispatch is so nice and tidy, that it makes us want to use it more broadly.

Again, this is not possible "natively" in Julia. But thanks to Julia's metaprogramming capabilities,
there are some packages that extend the dispatching. Let's take a look at two packages specifically,
the `WhereTraits.jl` and the `KeywordDispatch.jl`.

First, let's use the `WhereTraits.jl` to create new functions to price `Board` types.

In [116]:
using WhereTraits

@traits function bestprice(b::Board, units) where {units == "m"}
    return b.boardtype == "whiteboard" ? reduce(*,b.size, init=10.5) : reduce(*,b.size, init=10.3)
end
@traits function bestprice(b::Board, units) where {units == "cm"}
    return b.boardtype == "whiteboard" ? reduce(*,b.size, init=10.5/(100^2)) : reduce(*,b.size/100, init=10.3/(100^2))
end

Of course, the example above if overkill, and we'd be better of with a simple "if". But you get the idea. 

The next example consists in dispatching based on keywords.

In [118]:
using KeywordDispatch



## *Appendix* 
#### Creating a linked list.

Let's create a linked list. Our list is a nested struct. To create it, we write a `mklist` function.

In [57]:
# Straight from https://github.com/ninjaaron/dispatching-design-patterns
struct Nil end
struct List{T}
    head::T
    tail::Union{List{T}, Nil}
end

mklist(array::AbstractArray{T}) where T = foldr(List{T}, array, init=Nil())

# implement the iteration protocol
Base.iterate(l::List) = iterate(l, l)
Base.iterate(::List, l::List) = l.head, l.tail
Base.iterate(::List, ::Nil) = nothing

list = mklist(1:5)
for val in list
    println(val)
end

1
2
3
4
5


----
### *References*

This tutorial draws heavily from [this talk](https://www.youtube.com/watch?v=n-E-1-A_rZM).